In [1]:
import pandas as pd
df = pd.read_csv('../data/recall_mood_data.csv', index_col=0)
df.head(1)

,Date,Experimenter,Subject Initials,SubjectID,ConditionExpt,ConditionSub,Credit,Notes,convos,recalls
0,2018-11-30 00:00:00,Evgeniia,JM,1,Incongruent,Happy,1,NaN,I'm Spencer by the way I'm Jake Nice to meet y...,We can start with living in Nashville I live ...


In [62]:
from tqdm.auto import tqdm
import evaluate
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

In [56]:

references = df['convos']
predictions = df['recalls']

bleu_scores = []
rouge_scores = []
for ref, pred in tqdm(zip(df['convos'], df['recalls'])):
    bleu_scores.append(bleu.compute(references=[ref], predictions=[pred]))
    rouge_scores.append(rouge.compute(references=[ref], predictions=[pred]))

0it [00:00, ?it/s]

In [48]:
df = pd.concat([df.reset_index(drop=True), pd.Dataframe(rouge_scores), pd.DataFrame(bleu_scores)], axis=1)

In [50]:
df.head(1)

,Date,Experimenter,Subject Initials,SubjectID,ConditionExpt,ConditionSub,Credit,Notes,convos,recalls,rouge1,rouge2,rougeL,rougeLsum,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,2018-11-30 00:00:00,Evgeniia,JM,1,Incongruent,Happy,1,NaN,I'm Spencer by the way I'm Jake Nice to meet y...,We can start with living in Nashville I live ...,0.389744,0.215555,0.262722,0.262722,0.013077,"[0.7904929577464789, 0.37389770723104054, 0.14...",0.060746,0.263085,568,2159


# No correlation

In [95]:
from scipy import stats

stats.spearmanr(df['ConditionExpt'].to_ca, df['rouge1'])

TypeError: dtype '<class 'pandas.core.arrays.categorical.Categorical'>' not understood

# Recall convos using maximal scores

We recall about half of the conversations using this method.

This is not working as well as I had imagined.

In [72]:
import numpy as np

for i, pred in enumerate(df['recalls']):
    results = []
    for ref in df['convos']:
        res = bleu.compute(references=[ref],
                            predictions=[pred],
                           )
        results.append(res['bleu'])
    most_similar = np.argmax(results)
    if most_similar == i or most_similar == (i//2)*2:
        print('Successful recall!')
    else:
        print(f'{i} was most similar to {most_similar}')

0
Successful recall!
0
Successful recall!
2
Successful recall!
92
3 was most similar to 92
4
Successful recall!
92
5 was most similar to 92
12
6 was most similar to 12
92
7 was most similar to 92
8
Successful recall!
8
Successful recall!
92
10 was most similar to 92
92
11 was most similar to 92
12
Successful recall!
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jovyan/conda_envs/transformers/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_211/956206569.py", line 6, in <module>
    res = bleu.compute(references=[ref],
  File "/home/jovyan/conda_envs/transformers/lib/python3.10/site-packages/evaluate/module.py", line 444, in compute
    output = self._compute(**inputs, **compute_kwargs)
  File "/home/jovyan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76/bleu.py", line 122, in _compute
    score = compute_bleu(
  File "/home/jovyan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76/nmt_bleu.py", line 75, in compute_bleu
    merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
  File "/home/jovyan/co

In [ ]:
import numpy as np

for i, pred in enumerate(df['recalls']):
    results = []
    for ref in df['convos']:
        res = rouge.compute(references=[ref],
                            predictions=[pred],
                            # use_aggregator=False,
                            # use_stemmer=True,
                            rouge_types=['rouge2']
                           )
        results.append(res['rouge2'])
    most_similar = np.argmax(results)
    print(f'{most_similar:<2}: ', end='')
    if most_similar == i or most_similar == (i//2)*2:
        print('Successful recall!')
    else:
        print(f'{i} was most similar to {most_similar}')

0 : Successful recall!
0 : Successful recall!
2 : Successful recall!
2 : Successful recall!
4 : Successful recall!
4 : Successful recall!
6 : Successful recall!
6 : Successful recall!
8 : Successful recall!
8 : Successful recall!
92: 10 was most similar to 92
10: Successful recall!
12: Successful recall!
12: Successful recall!
14: Successful recall!
14: Successful recall!
16: Successful recall!
